In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle,os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras import regularizers, optimizers
import sys, os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
#plt.rcParams['figure.figsize'] = [6.32, 5.11]
#plt.rcParams['figure.constrained_layout.use'] = True

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.linewidth'] = 3
plt.rcParams['axes.labelpad'] = 15

plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['xtick.major.size'] = 8
plt.rcParams['xtick.major.width'] = 3

plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['ytick.major.size'] = 8
plt.rcParams['ytick.major.width'] = 3

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.markerscale'] = 1

plt.rcParams['lines.markersize'] = 9
plt.rcParams['lines.linewidth'] = 2

In [ ]:
path = '../Data/'

In [ ]:
L = 40
temp_val = np.arange(0.25, 4.0001, 0.25)
temp_samples = 10000
total_samples = temp_samples * temp_val.shape[0]
test_size = 0.1
val_size = 0.2

In [ ]:
split_tc = True

In [ ]:
data = pickle.load(open(path + 'Ising2DFM_reSample_L40_T=All.pkl','rb'))
data = np.unpackbits(data).reshape(-1, L * L).astype(int)
data[np.where(data==0)]=-1

labels = pickle.load(open(path + 'Ising2DFM_reSample_L40_T=All_labels.pkl','rb'))

In [ ]:
first_tc = np.where(temp_val <= 2.)[0][-1] * temp_samples
last_tc = np.where(temp_val >= 3)[0][1] * temp_samples

In [ ]:
temp_labels = np.hstack([np.repeat(temp, temp_samples) for temp in temp_val])

In [ ]:
if split_tc:
    
    X_ordered = data[:first_tc, :]
    Y_ordered = np.column_stack((labels[:first_tc], temp_labels[:first_tc]))

    X_critical = data[first_tc:last_tc, :]
    Y_critical = np.column_stack((labels[first_tc:last_tc], temp_labels[first_tc:last_tc]))

    X_disordered = data[last_tc:, :]
    Y_disordered = np.column_stack((labels[last_tc:], temp_labels[last_tc:]))

    X = np.concatenate((X_ordered, X_disordered))
    Y = np.concatenate((Y_ordered, Y_disordered))
    
    del X_ordered, Y_ordered, X_disordered, Y_disordered, data, labels

else:
    
    X = data
    Y = np.column_stack((labels, temp_labels))
    
    del data, labels

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=val_size)

del X, Y

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], L, L, 1))
X_val = np.reshape(X_val, (X_val.shape[0], L, L, 1))
X_critical = np.reshape(X_critical, (X_critical.shape[0], L, L, 1))

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(2, 2), activation='relu', input_shape=(L, L, 1), 
         data_format='channels_last'))

#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu'))

#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.5))
          
model.add(Flatten())

model.add(Dense(64, activation='relu'))
          
#model.add(Dropout(0.5))
          
model.add(Dense(1, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, Y_train[:, 0], validation_data=(X_val, Y_val[:, 0]) , batch_size=64, epochs=5)

In [ ]:
#model = Sequential()

#model.add(Conv2D(filters=10, kernel_size=(5, 5), strides=(1, 1), padding='valid', activation='relu', input_shape=(L,L,1), 
#         data_format='channels_last'))

#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(filters=20, kernel_size=(5, 5), activation='sigmoid'))

#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.5))
          
#model.add(Flatten())

#model.add(Dense(64, activation='relu'))
          
#model.add(Dropout(0.5))
          
#model.add(Dense(1, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))

In [ ]:
Conv2D?